Get started with TensorBoard

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

In [3]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'), #menambahkan layer yg fully connected
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax') #inisiasi layer output yg hanya berupa 1 node krn termasuk dalam binary classification
  ])

Using TensorBoard with Keras Model.fit()

In [5]:
model = create_model()
model.compile(optimizer='adam', #kompilasi
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
#keras model fit
model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 21s 9ms/step - loss: 0.2176 - accuracy: 0.9347 - val_loss: 0.1103 - val_accuracy: 0.9657
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0937 - accuracy: 0.9715 - val_loss: 0.0776 - val_accuracy: 0.9769
Epoch 3/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0682 - accuracy: 0.9783 - val_loss: 0.0714 - val_accuracy: 0.9783
Epoch 4/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0532 - accuracy: 0.9826 - val_loss: 0.0745 - val_accuracy: 0.9771
Epoch 5/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0441 - accuracy: 0.9855 - val_loss: 0.0730 - val_accuracy: 0.9790


In [6]:
%tensorboard --logdir logs/fit

Using TensorBoard with other methods

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [9]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [10]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [11]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [12]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24561071395874023, Accuracy: 92.80166625976562, Test Loss: 0.10624506324529648, Test Accuracy: 96.77000427246094
Epoch 2, Loss: 0.10312759131193161, Accuracy: 96.89666748046875, Test Loss: 0.07783098518848419, Test Accuracy: 97.53999328613281
Epoch 3, Loss: 0.07311956584453583, Accuracy: 97.78666687011719, Test Loss: 0.07429265975952148, Test Accuracy: 97.75
Epoch 4, Loss: 0.05467051640152931, Accuracy: 98.32500457763672, Test Loss: 0.06442072242498398, Test Accuracy: 98.06999969482422
Epoch 5, Loss: 0.04272298514842987, Accuracy: 98.63500213623047, Test Loss: 0.06277037411928177, Test Accuracy: 98.00999450683594


In [13]:
%tensorboard --logdir logs/gradient_tape

TensorBoard.dev: Host and share your ML experiment results

In [ ]:
!tensorboard dev upload \
  --logdir logs/fit \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot